In [1]:
%load_ext cython
%matplotlib inline
import sys
sys.path.append('/home/bram/ANTS')
sys.path.append('/home/bram/ANTS/entropy')
for p in sys.path:
    print(p)


/usr/lib/python36.zip
/usr/lib/python3.6
/usr/lib/python3.6/lib-dynload
/home/bram/PythonEnvs/Ants/lib/python3.6/site-packages
/home/bram/PythonEnvs/Ants/lib/python3.6/site-packages/IPython/extensions
/home/bram/.ipython
/home/bram/ANTS
/home/bram/ANTS/entropy


In [8]:
%%cython -a
# distutils: language = c++

import numpy as np

from cythonic.core.queen cimport Queen
from cythonic.core.ant cimport Ant, ant_state
from cythonic.core.domain cimport Domain
from cythonic.plugins.dummy_dicts import ant_dict,domain_dict, queen_dict, sim_dict, deposit_dict, gauss_dict
from cythonic.plugins.positions cimport point
from cythonic.plugins.dep_functions cimport dep_constant, dep_fun_args
# ctypedef readonly void (*f_dep)(double * x, ant_state* s, dep_fun_args* args)
from cythonic.core.ant cimport f_dep, ant_state
cimport cython


import matplotlib.pyplot as plt

from cythonic.plugins.sens_structs cimport  observations
cdef void check(point * p, observations * q):
    print(p[0])
    print(q[0])


    
def new_positions(deploy_style, n, **kwargs):
    xy = np.zeros([n,2], dtype = np.float)
    tetas = np.zeros(n,dtype=np.float)
    
    if deploy_style =='nest_radian':
        " all ants start aligned with a radian originating from the nest"
        x = kwargs['nest_loc']['x']
        y = kwargs['nest_loc']['y']
        R = kwargs['R']
        for i in range(n): 
            # expect nest location [nest_loc] and radius [R] in kwargs
            teta = np.random.rand()*2*np.pi
            xy[i,0] = x+np.cos(teta)*R
            xy[i,1] = y+np.sin(teta)*R
            tetas[i] = teta*180/np.pi
            
    return xy, tetas
            
def deploy_times(deploy_method, n, **kwargs):
    " generate a list of time instances when an ant is to be deployed "
    if deploy_method == 'instant':
        " deploy all at time t==0 "
        return np.zeros(n,dtype = np.float)
    elif deploy_method == 'uniform_dist':
        " deploy times uniformly distributed between 0 and kwargs['t_max']"
        t = kwargs['t_max']*np.random.rand(n)
    elif deploy_method == 'gamma_dist':
        """ deploy times gamma distribution 
        (https://docs.scipy.org/doc/numpy/reference/generated/numpy.random.gamma.html)
        shape k = kwargs['k'], scale theta (inverse of 'steepness' of the pdf) = kwargs['teta'] 
        median at aprox k*theta distr"""
        t = np.random.gamma(size = n,shape = kwargs['k'],scale=kwargs['teta'] )
    return np.sort(t-np.min(t))
    
@cython.boundscheck(False)
@cython.embedsignature(False)
@cython.wraparound(False)
@cython.nonecheck(False)
@cython.cdivision(True)
@cython.initializedcheck(False)
# @cython.profile(True)
cdef class Sim:
    cdef Queen queen
    cdef Domain domain
    
    cdef:
        readonly bint deployed #flag: all ants active == True
        readonly double[:] deploy_times #list of timers when ant should become active
        readonly double dt #simulation time step
        readonly double t #simulation time
        public unsigned int foodcount
        public unsigned int nestcount
        public unsigned int steps
    
    def __cinit__(self, queen_args, domain_args, unsigned int n_agents, double dt, unsigned int steps, deploy_style, deploy_timing,deploy_timing_args ):
        
        " bring the controller online "
        self.queen = Queen(n = n_agents, **queen_args)
        
        " initialize the domain"
        self.domain = Domain(**domain_args)
                
        " set simulation variables "
        self.t = 0.
        self.dt = dt
        self.foodcount = 0
        self.nestcount = 0
        self.steps = steps
        self.deploy_times = deploy_times(deploy_timing,n_agents, **deploy_timing_args)
        
        " initialize the ant states "
        xy, tetas = new_positions(deploy_style, n_agents,**{'R':self.domain.nest_radius, 'nest_loc':self.domain.nest_location})        
        self.queen.initialize_states(xy,tetas)
        
        "activate ants for the first time step if necessary"
        self.expand_active()
        print('Active ants: ',self.queen.count_active)
        
        
    """ ===================== Additional initializations ===================== """
    cdef readonly void set_depositing(self, str fun_type, dep_fun_args):
        " setup the functions for dropping pheromone "
        self.queen.setup_ant_depositing(fun_type, dep_fun_args )
        
    def set_gaussian(self, double covariance, double significancy):
        " make sure there is a nice gaussian meshmap representing a single deposition of pheromone "
        self.domain.init_gaussian(sigma = covariance,significancy = significancy)
        
        
    """ ===================== Actual simulation ===================== """
    cdef readonly void sim_step(self):
        " do a step in the simulation, perform all simulation logic (boundary check e.g..)"
        cdef unsigned int i
        
        " do a round of pheromone depositing prior to stepping"
        for i in range(self.queen.count_active):
            self.domain.add_pheromone(p = &self.queen.state_list[i].pos, Q = &self.queen.drop_quantity[i])
            # deposit the pheromone
            
        for i in range(self.queen.count_active):
            #target ant
            self.queen.assign_state(&i)
            
            #sense (queen/domain)
            self.domain.fill_observations(&self.queen.pheromone_vec[i], &self.queen.state_list[i].left, &self.queen.state_list[i].right)
                        
            #step (queen)
            self.queen.step(&self.dt)        

            #check conditions (boundary, nest and food)
            self.check_target()
            
            #poll the deposition quantity
            self.queen.agent.calc_quantity(&self.queen.drop_quantity[i])
            
        #evaporate (domain)
        
        
    cdef void expand_active(self):
        " check if new agents are to be activated based on simulation time "
        cdef unsigned int i
        for i in range(self.queen.count_active, self.queen.n,1):
            if self.deploy_times[i] <= self.t:
                " deploy ant if its deploy time is  \seq (<=) than current time step"
                self.queen.deploy(i) # deploy ant by id
            
        
    cdef readonly void check_target(self):
        """ Check the following:
            - Ant is out of bound? correct position
            - Ant is foodbound and at food? make nestbound and reverse
            - Ant is nestbound and at nest? make foodbound and reverse
        """
        " found food! "
        if self.queen.agent.state[0].foodbound and (
             self.domain.check_pos(&self.queen.agent.state.left, False) or
             self.domain.check_pos(&self.queen.agent.state.right, False)):
            self.queen.agent.reverse()
            self.foodcount+=1
            self.queen.agent.state[0].time = 0.
            
            " count!"
            " found nest! "
        elif (not self.queen.agent.state[0].foodbound) and (
            self.domain.check_pos(&self.queen.agent.state.left, True) or
            self.domain.check_pos(&self.queen.agent.state.right, True)):
            self.queen.agent.reverse()
            self.nestcount += 1
            self.queen.agent.state[0].time = 0.
            
        if not self.domain.check_bounds(&self.queen.agent.state.pos):
            " ant is out of bounds "
            self.domain.constraint(&self.queen.agent.state.pos)
            self.queen.agent.out_of_bounds(1)
            self.queen.agent.set_sensors()
            
        else:
            " make sure out_of_bounds flag is false "
            self.queen.agent.out_of_bounds(0)

sim_dict['n_agents'] = 100
gauss_dict['covariance'] = 5
cdef Sim sim = Sim(queen_args = queen_dict, domain_args = domain_dict, **sim_dict)
sim.set_depositing('constant', deposit_dict)
sim.set_gaussian(**gauss_dict)
sim.t+=sim.dt
sim.expand_active()
print('Active ants: ', sim.queen.count_active)

sim.sim_step()

cdef f_dep f = dep_constant
cdef dep_fun_args args = deposit_dict
cdef ant_state s = sim.queen.state_list[0]

cdef double q = 0.
print(args)
print(s)
f(&q, &s,&args)
print(q)

print(np.array(sim.queen.drop_quantity))
print(sim.domain.Gaussian.max())


Active ants:  1
Active ants:  3
{'q': 1.0, 'beta': 0.2, 'return_factor': 1.0}
{'id': 0, 'pos': {'x': 202.60015325921555, 'y': 527.897213034931}, 'left': {'x': 197.75945590374684, 'y': 526.6451550838927}, 'right': {'x': 201.3480953081772, 'y': 532.7379103903997}, 'theta': 149.5018693026051, 'omega': -0.021077939637778594, 'v': 5.0, 'Q_obs': {'lft': 1.0036913874411904, 'rght': 1.0040592668874062}, 'foodbound': True, 'active': True, 'out_of_bounds': False, 'rng_timer': 0.0, 'time': 1.0}
1.0
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1.]
0.006366197723675813


In [9]:
%%cython -a
import numpy as np

cdef double[::,:] xy = np.array([[1.,2.],[3.,4.]])
print(xy[0,0])
print(xy[1,0])
print(np.random.rand(10))

cdef struct tmp:
    double i
    double j
    double k

cdef double dummy_fun(double * x):
    x[0] = 2.
    
cdef double[:] ones = np.ones(10, dtype = np.float)
dummy_fun(&ones[1])
print(np.array(ones))
cdef bytes teststring = b'hoi'
print(teststring)


x = deploy_times('gamma_dist', n=1000, teta = 2,k= 4)
plt.hist(x)
plt.show()
print(np.array(sim.deploy_times))
print(f'median: {np.median(x)}, max: {np.max(x)}')

1.0
3.0
[0.18485284 0.43695725 0.99123812 0.20446679 0.34322056 0.49242898
 0.7886742  0.31558384 0.24715059 0.3033761 ]
[1. 2. 1. 1. 1. 1. 1. 1. 1. 1.]
b'hoi'
{'q': 1.0, 'beta': 0.2, 'return_factor': 1.0}
